In [ ]:
import pandas as pd
import numpy as np

# Import the first CSV file
fights_df = pd.read_csv('ufc_fights_data.csv')

# Import the second CSV file
fighters_df = pd.read_csv('ufc_fighters_data.csv')
pd.set_option('display.max_columns', None)
fighters_df.drop('url', axis=1, inplace=True) 


In [ ]:
#clean fighters_df

# Replace '--' with NaN across the whole DataFrame
fighters_df.replace('--', np.nan, inplace=True)

# Remove leading colons
cols_to_clean = ['SLpM', 'Str. Acc.', 'SApM', 'Str. Def','TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.']
for col in cols_to_clean:
  fighters_df[col] = fighters_df[col].astype(str).str.lstrip(':')

#clean weight, reach, and height
fighters_df['weight'] = fighters_df['weight'].astype(str).str.replace('lbs.', '', regex=False).str.strip()
fighters_df['weight'] = pd.to_numeric(fighters_df['weight'], errors='coerce')

fighters_df['reach'] = pd.to_numeric(
    fighters_df['reach'].astype(str).str.replace('"', '', regex=False).str.strip(),
    errors='coerce'
)

def parse_height(h):
    if isinstance(h, str) and "'" in h:
        ft, inch = h.split("'")
        return int(ft.strip()) * 12 + int(inch.strip().replace('"', ''))
    return None

fighters_df['height'] = fighters_df['height'].apply(parse_height)


#change percents to decimals
percent_cols = ['Str. Acc.', 'Str. Def', 'TD Acc.', 'TD Def.']
for col in percent_cols:
    fighters_df[col] = (
        fighters_df[col]
        .astype(str)
        .str.replace('%', '', regex=False)  # Remove %
        .str.strip()                        # Trim whitespace
        .astype(float) / 100                # Convert to decimal
    )


#format dob
fighters_df['dob'] = pd.to_datetime(fighters_df['dob'], errors='coerce')


#turn record collumn to wins, losses, draws, nc
fighters_df[['wins', 'losses', 'draws', 'nc']] = fighters_df['record'].str.extract(
    r'^(\d+)-(\d+)(?:-(\d+))?(?:\s*\((\d+)\s*NC\))?$'
)
cols = ['wins', 'losses', 'draws', 'nc']
fighters_df[cols] = fighters_df[cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

fighters_df.head(10)
fighters_df.to_csv('ufc_fighters_cleaned.csv', index=False)

In [ ]:
#clean fights_df

#replace '--' values and turn percents to decimals
percent_cols = ['p1_SIG_STR_PCT', 'p2_SIG_STR_PCT', 'p1_TD_PCT', 'p2_TD_PCT', 'p1_R1_SIG_STR_PCT', 'p2_R1_SIG_STR_PCT', 'p1_R1_TD_PCT', 'p2_R1_TD_PCT', 'p1_SIG_STR_PCT_DETAILED', 'p2_SIG_STR_PCT_DETAILED', 'p1_R1_SIG_STR_PCT_DETAILED', 'p2_R1_SIG_STR_PCT_DETAILED']
for col in percent_cols:
    fights_df[col] = (
        fights_df[col]
        .astype(str)
        .replace({'--': np.nan, '---': np.nan})  # Handle missing values first
        .str.replace('%', '', regex=False)  # Remove percentage symbols
        .str.strip()  # Clean whitespace
        .astype(float) / 100  # Convert to decimal
    )


#format event date
fights_df['event_date'] = pd.to_datetime(fights_df['event_date'], errors='coerce')


# columns to split
cols_to_split = ['p1_SIG_STR', 'p2_SIG_STR', 'p1_TOTAL_STR', 'p2_TOTAL_STR', 'p1_TD', 'p2_TD', 'p1_R1_SIG_STR', 'p2_R1_SIG_STR', 'p1_R1_TOTAL_STR', 'p2_R1_TOTAL_STR', 'p1_R1_TD', 'p2_R1_TD', 'p1_SIG_STR_LANDED_ATTEMPTED', 'p2_SIG_STR_LANDED_ATTEMPTED', 'p1_HEAD_LANDED_ATTEMPTED', 'p2_HEAD_LANDED_ATTEMPTED', 'p1_BODY_LANDED_ATTEMPTED', 'p2_BODY_LANDED_ATTEMPTED', 'p1_LEG_LANDED_ATTEMPTED', 'p2_LEG_LANDED_ATTEMPTED', 'p1_DISTANCE_LANDED_ATTEMPTED', 'p2_DISTANCE_LANDED_ATTEMPTED', 'p1_CLINCH_LANDED_ATTEMPTED', 'p2_CLINCH_LANDED_ATTEMPTED', 'p1_GROUND_LANDED_ATTEMPTED', 'p2_GROUND_LANDED_ATTEMPTED', 'p1_R1_SIG_STR_LANDED_ATTEMPTED', 'p2_R1_SIG_STR_LANDED_ATTEMPTED', 'p1_R1_HEAD_LANDED_ATTEMPTED', 'p2_R1_HEAD_LANDED_ATTEMPTED', 'p1_R1_BODY_LANDED_ATTEMPTED', 'p2_R1_BODY_LANDED_ATTEMPTED', 'p1_R1_LEG_LANDED_ATTEMPTED', 'p2_R1_LEG_LANDED_ATTEMPTED', 'p1_R1_DISTANCE_LANDED_ATTEMPTED', 'p2_R1_DISTANCE_LANDED_ATTEMPTED', 'p1_R1_CLINCH_LANDED_ATTEMPTED', 'p2_R1_CLINCH_LANDED_ATTEMPTED', 'p1_R1_GROUND_LANDED_ATTEMPTED', 'p2_R1_GROUND_LANDED_ATTEMPTED']

#Loop over them, split into “_LANDED” and “_ATTEMPTED”, convert to ints
for col in cols_to_split:
    # decide new column names
    if col.endswith('_LANDED_ATTEMPTED'):
        base = col.rsplit('_', 2)[0]       # turns 'p1_HEAD_LANDED_ATTEMPTED' → 'p1_HEAD'
        landed_col    = f"{base}_LANDED"
        attempted_col = f"{base}_ATTEMPTED"
    else:
        # for columns like 'p1_SIG_STR'
        landed_col    = f"{col}_LANDED"
        attempted_col = f"{col}_ATTEMPTED"

    # split “63 of 129” → ['63','129'], then to numeric
    fights_df[[landed_col, attempted_col]] = (
        fights_df[col]
          .str.split(' of ', expand=True)
          .astype(int)
    )

# Drop the originals
fights_df.drop(columns=cols_to_split, inplace=True)


# time collumns needing reformatting
time_cols = ['p1_CTRL', 'p2_CTRL', 'p1_R1_CTRL', 'p2_R1_CTRL' ]

#Convert each to total seconds
for col in time_cols:
    # Replace non-time strings with NaN
    fights_df[col] = fights_df[col].replace({'--': np.nan, '---': np.nan})
    
    # Convert to "00:MM:SS" format then to seconds
    fights_df[col] = (
        pd.to_timedelta('00:' + fights_df[col].fillna('00:00'), errors='coerce')
        .dt.total_seconds()
        .astype('Int64')  # Allows integer NaNs
    )

fights_df.head(10)

,p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,method,referee,winner,event_date,p1_SIG_STR_LANDED,p1_SIG_STR_ATTEMPTED,p2_SIG_STR_LANDED,p2_SIG_STR_ATTEMPTED,p1_TOTAL_STR_LANDED,p1_TOTAL_STR_ATTEMPTED,p2_TOTAL_STR_LANDED,p2_TOTAL_STR_ATTEMPTED,p1_TD_LANDED,p1_TD_ATTEMPTED,p2_TD_LANDED,p2_TD_ATTEMPTED,p1_R1_SIG_STR_LANDED,p1_R1_SIG_STR_ATTEMPTED,p2_R1_SIG_STR_LANDED,p2_R1_SIG_STR_ATTEMPTED,p1_R1_TOTAL_STR_LANDED,p1_R1_TOTAL_STR_ATTEMPTED,p2_R1_TOTAL_STR_LANDED,p2_R1_TOTAL_STR_ATTEMPTED,p1_R1_TD_LANDED,p1_R1_TD_ATTEMPTED,p2_R1_TD_LANDED,p2_R1_TD_ATTEMPTED,p1_HEAD_LANDED,p1_HEAD_ATTEMPTED,p2_HEAD_LANDED,p2_HEAD_ATTEMPTED,p1_BODY_LANDED,p1_BODY_ATTEMPTED,p2_BODY_LANDED,p2_BODY_ATTEMPTED,p1_LEG_LANDED,p1_LEG_ATTEMPTED,p2_LEG_LANDED,p2_LEG_ATTEMPTED,p1_DISTANCE_LANDED,p1_DISTANCE_ATTEMPTED,p2_DISTANCE_LANDED,p2_DISTANCE_ATTEMPTED,p1_CLINCH_LANDED,p1_CLINCH_ATTEMPTED,p2_CLINCH_LANDED,p2_CLINCH_ATTEMPTED,p1_GROUND_LANDED,p1_GROUND_ATTEMPTED,p2_GROUND_LANDED,p2_GROUND_ATTEMPTED,p1_R1_HEAD_LANDED,p1_R1_HEAD_ATTEMPTED,p2_R1_HEAD_LANDED,p2_R1_HEAD_ATTEMPTED,p1_R1_BODY_LANDED,p1_R1_BODY_ATTEMPTED,p2_R1_BODY_LANDED,p2_R1_BODY_ATTEMPTED,p1_R1_LEG_LANDED,p1_R1_LEG_ATTEMPTED,p2_R1_LEG_LANDED,p2_R1_LEG_ATTEMPTED,p1_R1_DISTANCE_LANDED,p1_R1_DISTANCE_ATTEMPTED,p2_R1_DISTANCE_LANDED,p2_R1_DISTANCE_ATTEMPTED,p1_R1_CLINCH_LANDED,p1_R1_CLINCH_ATTEMPTED,p2_R1_CLINCH_LANDED,p2_R1_CLINCH_ATTEMPTED,p1_R1_GROUND_LANDED,p1_R1_GROUND_ATTEMPTED,p2_R1_GROUND_LANDED,p2_R1_GROUND_ATTEMPTED
0,Cory Sandhagen,Deiveson Figueiredo,0,0,0.62,0.66,1.00,0.40,0,0,1,1,307,31,0,0,0.79,1.00,1.00,0.25,0,0,0,0,217,4,0.62,0.66,0.79,1.00,KO/TKO,Dan Miragliotta,1,2025-05-03,54,87,6,9,92,149,8,11,2,2,2,5,31,39,2,2,53,78,3,3,1,1,1,4,33,65,4,6,2,2,0,0,19,20,2,3,24,43,4,7,0,0,2,2,30,44,0,0,22,30,1,1,1,1,0,0,8,8,1,1,10,13,1,1,0,0,1,1,21,26,0,0
1,Reinier de Ridder,Bo Nickal,1,0,0.68,0.38,0.00,1.00,0,0,0,0,129,144,0,0,0.50,0.33,0.00,1.00,0,0,0,0,112,120,0.68,0.38,0.50,0.33,KO/TKO,Mike Beltran,1,2025-05-03,22,32,8,21,91,104,24,40,0,3,1,1,5,10,2,6,43,49,17,24,0,2,1,1,9,18,2,13,13,14,4,5,0,0,2,3,7,15,3,14,15,17,5,7,0,0,0,0,2,7,1,5,3,3,1,1,0,0,0,0,1,5,0,4,4,5,2,2,0,0,0,0
2,Santiago Ponzinibbio,Daniel Rodriguez,0,1,0.44,0.39,0.12,NaN,0,0,0,0,44,7,0,0,0.42,0.43,0.00,NaN,0,0,0,0,2,7,0.44,0.39,0.42,0.43,KO/TKO,Kevin MacDonald,0,2025-05-03,49,111,66,166,50,115,70,170,1,8,0,0,15,35,35,81,15,35,35,81,0,2,0,0,31,83,39,131,6,16,20,25,12,12,7,10,49,111,64,164,0,0,1,1,0,0,1,1,5,19,20,61,3,9,13,16,7,7,2,4,15,35,34,80,0,0,1,1,0,0,0,0
3,Montel Jackson,Daniel Marcos,0,0,0.45,0.53,0.25,0.42,1,0,0,0,217,345,0,0,0.33,0.57,0.25,1.00,0,0,0,0,127,86,0.45,0.53,0.33,0.57,Decision - Unanimous,Gary Copeland,1,2025-05-03,30,66,17,32,67,106,24,43,2,8,3,7,4,12,4,7,22,31,7,11,1,4,1,1,25,58,5,18,1,3,5,7,4,5,7,7,27,62,13,27,3,4,3,4,0,0,1,1,2,8,2,4,1,2,1,2,1,2,1,1,3,11,3,6,1,1,1,1,0,0,0,0
4,Cameron Smotherman,Serhiy Sidey,0,0,0.43,0.41,NaN,0.41,0,1,0,0,0,252,0,0,0.51,0.54,NaN,0.00,0,0,0,0,0,55,0.43,0.41,0.51,0.54,Decision - Unanimous,Dan Miragliotta,0,2025-05-03,58,134,55,132,61,137,66,149,0,0,5,12,22,43,27,50,25,46,28,51,0,0,0,1,43,114,30,98,15,20,15,22,0,0,10,12,53,124,49,122,5,10,6,10,0,0,0,0,16,34,12,33,6,9,7,8,0,0,8,9,22,40,23,45,0,3,4,5,0,0,0,0
5,Jeremy Stephens,Mason Jones,0,0,0.38,0.35,NaN,0.66,0,0,0,0,11,363,0,0,0.44,0.37,NaN,1.00,0,0,0,0,0,43,0.38,0.35,0.44,0.37,Decision - Unanimous,Mike Beltran,0,2025-05-03,42,108,81,231,54,126,104,261,0,0,8,12,21,47,39,103,24,53,41,108,0,0,2,2,19,79,50,188,7,11,19,28,16,18,12,15,41,105,74,220,1,3,5,7,0,0,2,4,6,28,22,80,3,6,9,13,12,13,8,10,20,45,36,100,1,2,3,3,0,0,0,0
6,Yana Santos,Miesha Tate,0,0,0.56,0.34,0.00,0.14,0,0,0,0,46,305,0,0,0.50,

In [ ]:
# merge the two datasets

# First, merge p1 fighter stats
fights_df = fights_df.merge(
    fighters_df,
    left_on='p1_fighter',
    right_on='name',  # This matches p1_fighter with the name column in fighters_df
    how='left'
)
fights_df.drop(columns=['name'], inplace=True)

# Only include the specific columns you want to rename with p1_ prefix
p1_cols = ['height', 'weight', 'reach', 'stance', 'dob', 'SLpM', 'Str. Acc.', 'SApM', 'Str. Def', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.']
fights_df.rename(columns={col: f'p1_{col}' for col in p1_cols}, inplace=True)

# Then merge p2 fighter stats
fights_df = fights_df.merge(
    fighters_df,
    left_on='p2_fighter',
    right_on='name',  # This matches p2_fighter with the name column in fighters_df
    how='left'
)
fights_df.drop(columns=['name'], inplace=True)

# Use the same columns for p2 as we did for p1
p2_cols = ['height', 'weight', 'reach', 'stance', 'dob', 'SLpM', 'Str. Acc.', 'SApM', 'Str. Def', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.']
fights_df.rename(columns={col: f'p2_{col}' for col in p2_cols}, inplace=True)


# Drop unwanted columns
columns_to_drop = ['record_x', 'wins_x', 'losses_x', 'draws_x', 'nc_x', 'record_y', 'wins_y', 'losses_y', 'draws_y', 'nc_y']

# Check if columns exist before trying to drop them
existing_columns = [col for col in columns_to_drop if col in fights_df.columns]
if existing_columns:
    fights_df.drop(columns=existing_columns, inplace=True)

fights_df.head(10)

,p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,method,referee,winner,event_date,p1_SIG_STR_LANDED,p1_SIG_STR_ATTEMPTED,p2_SIG_STR_LANDED,p2_SIG_STR_ATTEMPTED,p1_TOTAL_STR_LANDED,p1_TOTAL_STR_ATTEMPTED,p2_TOTAL_STR_LANDED,p2_TOTAL_STR_ATTEMPTED,p1_TD_LANDED,p1_TD_ATTEMPTED,p2_TD_LANDED,p2_TD_ATTEMPTED,p1_R1_SIG_STR_LANDED,p1_R1_SIG_STR_ATTEMPTED,p2_R1_SIG_STR_LANDED,p2_R1_SIG_STR_ATTEMPTED,p1_R1_TOTAL_STR_LANDED,p1_R1_TOTAL_STR_ATTEMPTED,p2_R1_TOTAL_STR_LANDED,p2_R1_TOTAL_STR_ATTEMPTED,p1_R1_TD_LANDED,p1_R1_TD_ATTEMPTED,p2_R1_TD_LANDED,p2_R1_TD_ATTEMPTED,p1_HEAD_LANDED,p1_HEAD_ATTEMPTED,p2_HEAD_LANDED,p2_HEAD_ATTEMPTED,p1_BODY_LANDED,p1_BODY_ATTEMPTED,p2_BODY_LANDED,p2_BODY_ATTEMPTED,p1_LEG_LANDED,p1_LEG_ATTEMPTED,p2_LEG_LANDED,p2_LEG_ATTEMPTED,p1_DISTANCE_LANDED,p1_DISTANCE_ATTEMPTED,p2_DISTANCE_LANDED,p2_DISTANCE_ATTEMPTED,p1_CLINCH_LANDED,p1_CLINCH_ATTEMPTED,p2_CLINCH_LANDED,p2_CLINCH_ATTEMPTED,p1_GROUND_LANDED,p1_GROUND_ATTEMPTED,p2_GROUND_LANDED,p2_GROUND_ATTEMPTED,p1_R1_HEAD_LANDED,p1_R1_HEAD_ATTEMPTED,p2_R1_HEAD_LANDED,p2_R1_HEAD_ATTEMPTED,p1_R1_BODY_LANDED,p1_R1_BODY_ATTEMPTED,p2_R1_BODY_LANDED,p2_R1_BODY_ATTEMPTED,p1_R1_LEG_LANDED,p1_R1_LEG_ATTEMPTED,p2_R1_LEG_LANDED,p2_R1_LEG_ATTEMPTED,p1_R1_DISTANCE_LANDED,p1_R1_DISTANCE_ATTEMPTED,p2_R1_DISTANCE_LANDED,p2_R1_DISTANCE_ATTEMPTED,p1_R1_CLINCH_LANDED,p1_R1_CLINCH_ATTEMPTED,p2_R1_CLINCH_LANDED,p2_R1_CLINCH_ATTEMPTED,p1_R1_GROUND_LANDED,p1_R1_GROUND_ATTEMPTED,p2_R1_GROUND_LANDED,p2_R1_GROUND_ATTEMPTED,p1_height,p1_weight,p1_reach,p1_stance,p1_dob,p1_SLpM,p1_Str. Acc.,p1_SApM,p1_Str. Def,p1_TD Avg.,p1_TD Acc.,p1_TD Def.,p1_Sub. Avg.,p2_height,p2_weight,p2_reach,p2_stance,p2_dob,p2_SLpM,p2_Str. Acc.,p2_SApM,p2_Str. Def,p2_TD Avg.,p2_TD Acc.,p2_TD Def.,p2_Sub. Avg.
0,Cory Sandhagen,Deiveson Figueiredo,0,0,0.62,0.66,1.00,0.40,0,0,1,1,307,31,0,0,0.79,1.00,1.00,0.25,0,0,0,0,217,4,0.62,0.66,0.79,1.00,KO/TKO,Dan Miragliotta,1,2025-05-03,54,87,6,9,92,149,8,11,2,2,2,5,31,39,2,2,53,78,3,3,1,1,1,4,33,65,4,6,2,2,0,0,19,20,2,3,24,43,4,7,0,0,2,2,30,44,0,0,22,30,1,1,1,1,0,0,8,8,1,1,10,13,1,1,0,0,1,1,21,26,0,0,71.0,135.0,70.0,Switch,1992-04-20,5.02,0.44,3.46,0.56,1.19,0.32,0.63,0.3,65.0,135.0,68.0,Orthodox,1987-12-18,2.91,0.54,3.55,0.49,1.62,0.34,0.60,1.4
1,Reinier de Ridder,Bo Nickal,1,0,0.68,0.38,0.00,1.00,0,0,0,0,129,144,0,0,0.50,0.33,0.00,1.00,0,0,0,0,112,120,0.68,0.38,0.50,0.33,KO/TKO,Mike Beltran,1,2025-05-03,22,32,8,21,91,104,24,40,0,3,1,1,5,10,2,6,43,49,17,24,0,2,1,1,9,18,2,13,13,14,4,5,0,0,2,3,7,15,3,14,15,17,5,7,0,0,0,0,2,7,1,5,3,3,1,1,0,0,0,0,1,5,0,4,4,5,2,2,0,0,0,0,76.0,185.0,78.0,Southpaw,1990-09-07,3.28,0.57,2.69,0.38,6.89,0.46,1.00,2.0,73.0,185.0,76.0,Southpaw,1996-01-14,2.65,0.63,1.86,0.64,3.61,0.50,0.00,3.6
2,Santiago Ponzinibbio,Daniel Rodriguez,0,1,0.44,0.39,0.12,NaN,0,0,0,0,44,7,0,0,0.42,0.43,0.00,NaN,0,0,0,0,2,7,0.44,0.39,0.42,0.43,KO/TKO,Kevin MacDonald,0,2025-05-03,49,111,66,166,50,115,70,170,1,8,0,0,15,35,35,81,15,35,35,81,0,2,0,0,31,83,39,131,6,16,20,25,12,12,7,10,49,111,64,164,0,0,1,1,0,0,1,1,5,19,20,61,3,9,13,16,7,7,2,4,15,35,34,80,0,0,1,1,0,0,0,0,72.0,170.0,73.0,Orthodox,1986-09-26,4.80,0.41,4.61,0.61,0.56,0.32,0.71,0.0,73.0,170.0,74.0,Southpaw,1986-12-31,7.50,0.49,5.47,0.58,0.59,0.54,0.57,0.1
3,Montel Jackson,Daniel Marcos,0,0,0.45,0.53,0.25,0.42,1,0,0,0,217,345,0,0,0.33,0.57,0.25,1.00,0,0,0,0,127,86,0.45,0.53,0.33,0.57,Decision - Unanimous,Gary Copeland,1,2025-05-03,30,66,17,32,67,106,24,43,2,8,3,7,4,12,4,7,22,31,7,11,1,4,1,1,25,58,5,18,1,3,5,7,4,5,7,7,27,62,13,27,3,4,3,4,0,0,1,1,2,8,2,4,1,2,1,2,1,2,1,1,3,11,3,6,1,1,1,1,0,0,0,0,70.0,135.0,75.0,Southpaw,1992-04-24,3.39,0.53,1.42,0.63,3.41,0.71,0.69,0.3,67.0,135.0,69.0,Orthodo

In [5]:
fights_df.to_csv('ufc_cleaned.csv', index=False)